In [71]:
%pylab inline

from tqdm import tqdm_notebook as tqdm
import requests
import pickle
import re
import itertools
import functools
import collections
import string
import time
from bs4 import BeautifulSoup
from sortedcontainers import SortedList

from adventlib import *

YEAR = 2021
DAY = int('15')

submit1, submit2 = generate_submits(YEAR, DAY)

while True:
    try:
        raw_input = get_input(YEAR, DAY)
        break
    except Exception as e:
        print(e)
        time.sleep(1)

Populating the interactive namespace from numpy and matplotlib


In [17]:
import heapq

In [32]:
lines, groups = linesplit("""
1163751742
1381373672
2136511328
3694931569
7463417111
1319128137
1359912421
3125421639
1293138521
2311944581
""".strip())

In [73]:
lines, groups = linesplit(raw_input, verbose=True)

100 lines
1 groups
>>>
9937913613489579689887899839718659949144483893874749858366681232699911196851919824197391598696712368
5253918396598241962985986992428814875993889428717557939529588827685169588381194992989482737895729941
4729891993659869351699195978189948897799346577498659736999269171689999888159836937748681169667242592
...


In [74]:
a = lines_as_matrix(lines)
a = vectorize(int)(a)
a

array([[9, 9, 3, ..., 3, 6, 8],
       [5, 2, 5, ..., 9, 4, 1],
       [4, 7, 2, ..., 5, 9, 2],
       ...,
       [6, 9, 9, ..., 3, 9, 2],
       [7, 5, 9, ..., 6, 9, 9],
       [6, 1, 9, ..., 9, 1, 9]])

In [75]:
n, m = a.shape
n, m

(100, 100)

In [76]:
d = [
    [-1, 0],
    [0, 1],
    [1, 0],
    [0, -1],
]

In [77]:
q = []
heapq.heappush(q, (0, 0, 0))
dp = zeros_like(a) + 1000000

while q:
    dist, i, j = heapq.heappop(q)
    if dist > dp[i, j]:
        continue

    for dd in d:
        ii = i + dd[0]
        jj = j + dd[1]
        if 0 <= ii < n and 0 <= jj < m:
            ddist = dist + a[ii, jj]
            if ddist < dp[ii, jj]:
                dp[ii, jj] = ddist
                heapq.heappush(q, (ddist, ii, jj))

In [78]:
dp

array([[ 14,   9,  12, ..., 513, 519, 527],
       [  5,   7,  12, ..., 521, 523, 524],
       [  9,  14,  14, ..., 512, 521, 523],
       ...,
       [476, 471, 480, ..., 714, 723, 725],
       [483, 476, 485, ..., 720, 729, 734],
       [483, 477, 486, ..., 729, 730, 739]])

In [79]:
dp[-1, -1]

739

In [80]:
submit1(dp[-1, -1])

Already submitted (2021, 15, 1, 739): True


True

In [81]:
b = zeros((n * 5, m * 5), dtype=int)
for i in range(5):
    for j in range(5):
        x = a
        for k in range(i + j):
            x = (x % 9) + 1
        b[i * n : i * n + n, j * m : j * m + m] = x
b

array([[9, 9, 3, ..., 7, 1, 3],
       [5, 2, 5, ..., 4, 8, 5],
       [4, 7, 2, ..., 9, 4, 6],
       ...,
       [1, 4, 4, ..., 2, 8, 1],
       [2, 9, 4, ..., 5, 8, 8],
       [1, 5, 4, ..., 8, 9, 8]])

In [82]:
n, m = b.shape
n, m

q = []
heapq.heappush(q, (0, 0, 0))
dp = zeros_like(b) + 1000000

while q:
    dist, i, j = heapq.heappop(q)
    if dist > dp[i, j]:
        continue

    for dd in d:
        ii = i + dd[0]
        jj = j + dd[1]
        if 0 <= ii < n and 0 <= jj < m:
            ddist = dist + b[ii, jj]
            if ddist < dp[ii, jj]:
                dp[ii, jj] = ddist
                heapq.heappush(q, (ddist, ii, jj))

In [83]:
dp

array([[  14,    9,   12, ..., 1737, 1738, 1741],
       [   5,    7,   12, ..., 1733, 1741, 1746],
       [   9,   14,   14, ..., 1737, 1736, 1742],
       ...,
       [1744, 1748, 1747, ..., 3015, 3023, 3024],
       [1746, 1755, 1748, ..., 3020, 3028, 3032],
       [1747, 1752, 1752, ..., 3028, 3037, 3040]])

In [84]:
submit2(dp[-1, -1])

Already submitted (2021, 15, 2, 3040): True


True

In [69]:
d = SortedList()
n = 1000000
m = 1000000000000
for i in range(n):
    d[np.random.randint(m)] = np.random.randint(m)

In [70]:
%%timeit
0 in d

48.8 ns ± 3.3 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [100]:
%%timeit
n, m = b.shape
n, m

q = []
heapq.heappush(q, (0, 0, 0))
dp = zeros_like(b) + 1000000
adds = 0

while q:
    dist, i, j = heapq.heappop(q)
    if dist > dp[i, j]:
        continue

    for dd in d:
        ii = i + dd[0]
        jj = j + dd[1]
        if 0 <= ii < n and 0 <= jj < m:
            ddist = dist + b[ii, jj]
            if ddist < dp[ii, jj]:
                dp[ii, jj] = ddist
                heapq.heappush(q, (ddist, ii, jj))
                adds += 1
print(adds)

250000
250000
250000
250000
250000
250000
250000
250000
1.39 s ± 105 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [104]:
%%timeit
n, m = b.shape
n, m

M = 1000000

q = SortedList()
q.add((0, 0, 0))
dp = zeros_like(b) + M
adds = 0

while q:
    dist, i, j = q.pop(0)

    for dd in d:
        ii = i + dd[0]
        jj = j + dd[1]
        if 0 <= ii < n and 0 <= jj < m:
            ddist = dist + b[ii, jj]
            if ddist < dp[ii, jj]:
#                 q.discard((dp[ii, jj], ii, jj))
                dp[ii, jj] = ddist
                q.add((dp[ii, jj], ii, jj))
                adds += 1
print(adds)

250000
250000
250000
250000
250000
250000
250000
250000
1.57 s ± 45.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
